# DAIN Colab

Credits to original Colab file: 
https://github.com/baowenbo/DAIN/issues/44

My fork:
https://github.com/styler00dollar/DAIN

Enhancement by [Styler00Dollar](https://github.com/styler00dollar) aka "sudo rm -rf / --no-preserve-root#8353" on discord and [Alpha](https://github.com/AlphaGit). Please do not run this command in your linux terminal. It's rather meant as a joke.

A simple guide:
- Copy the .ipynb-file to your drive.
- Create a folder inside of Google Drive named "DAIN"
- Change the configurations in the next cell
- Run cells one by one

Stuff that should be improved:
- Alpha channel will be removed automatically and won't be added back. Anything related to alpha will be converted to black.
- Adding configuration to select speed
- Detect scenes to avoid interpolating scene-changes
- Auto-resume
- Copy `start_frame` - `end_frame` audio from original input to final output

In [0]:
################# Configurations ############################

# Use the values in here to configure what you'd like DAIN to do.

# Input file: Path (relative to the root of your Google Drive) to the input file.
# For instance, if you save your "example.mkv" file in your Google Drive, inside a "videos" folder, the path would be:
# videos/example.mkv
INPUT_FILEPATH = "DAIN/input.mp4"

# Output file path: path (relative to the root of your Google Drive) for the output file. It will also determine the
# filetype in the destination. MP4 is recommended.
OUTPUT_FILE_PATH = "DAIN/output.mp4"

################# Optional configurations ############################

# Target FPS = how many frames per second should the result have. This will determine how many intermediate images are
# interpolated.
TARGET_FPS = 60

# Frame input directly
# Use a path that is in your GDrive if you already have the list of frames in the format 00001.png, 00002.png, etc.
# Your GDrive is located at `/content/gdrive/My Drive/`
FRAME_INPUT_DIR = '/content/DAIN/input_frames'

# Frame output directory
# Use a location in your GDrive if you want the generated frames stored to your Google Drive.
# Your GDrive is located at `/content/gdrive/My Drive/DAIN/tmp`
FRAME_OUTPUT_DIR = '/content/DAIN/output_frames'

# Seamless playback
# Creates a seamless loop by using the first frame as last one as well. Set this to True this if loop is intended.
SEAMLESS=False

In [0]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
print('Google Drive connected.')

In [0]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

# P100: 16GB (Works)
# T4: 16GB [RuntimeError: CUDA call failed]
# P4: 8GB (Works)
# K80: 8GB (Not tested)

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

# Install dependencies.

This next step may take somewhere between 15-20 minutes. Run this only once at startup.


In [0]:
from IPython.display import clear_output
!git clone https://github.com/styler00dollar/Colab-DAIN /content/DAIN

# This takes a while. Just wait. ~15 minutes.
# Building DAIN.
%cd /content/DAIN/my_package/
!./build.sh
print("Building #1 done.")

# Wait again. ~5 minutes.
# Building DAIN PyTorch correlation package.
%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
print("Building #2 done.")

# Downloading pre-trained model
%cd /content/DAIN
!mkdir model_weights
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

!CUDA_VISIBLE_DEVICES=0

!sudo apt-get install imagemagick imagemagick-doc

In [0]:
# Detecting FPS of input file.
%shell yes | cp -f /content/gdrive/My\ Drive/{INPUT_FILEPATH} /content/DAIN/

import os
filename = os.path.basename(INPUT_FILEPATH)

import cv2
cap = cv2.VideoCapture(f'/content/DAIN/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)

# ffmpeg extract - Generating individual frame PNGs from the source file.
%shell rm -rf '{FRAME_INPUT_DIR}'
%shell mkdir -p '{FRAME_INPUT_DIR}'

%shell ffmpeg -i '/content/DAIN/{filename}' '{FRAME_INPUT_DIR}/%05d.png'

png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
clear_output()

pngs_generated_count = int(png_generated_count_command_result.output.strip())

import shutil
if SEAMLESS==True:
    pngs_generated_count += 1
    original = str(FRAME_INPUT_DIR)+"/00001.png"
    target = str(FRAME_INPUT_DIR)+"/"+str(pngs_generated_count).zfill(5)+".png"
    shutil.copyfile(original, target)

print(f"Input FPS: {fps}")
print(f"{pngs_generated_count} frame PNGs generated.")

# Checking if PNG do have alpha
import subprocess as sp
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")

# Removing alpha if detected
if "a" in channels:
    print("Alpha detected. It will be removed.")
    print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))
  
%cd /content/DAIN

In [0]:
# Interpolation
%shell mkdir -p '{FRAME_OUTPUT_DIR}'
%cd /content/DAIN

!python colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {pngs_generated_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'

In [0]:
# Create video
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'

In [0]:
# [Experimental] Create video with sound
# Only run this, if the original had sound.
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -i '/content/DAIN/{filename}' -acodec copy output-audio.aac
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' -i output-audio.aac -shortest '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'